<a href="https://colab.research.google.com/github/Lee-gp/BI_projects/blob/master/Action2_DeepFM_Movielens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# !pip install deepctr
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat,VarLenSparseFeat,get_feature_names

In [9]:
#小样本数据加载
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir("/content/drive/My Drive/Colab Notebooks/BI_core/BI_core_L5")

data = pd.read_csv('movielens_sample.txt')
#所有特征作为稀疏特征
sparse_features = ['movie_id','user_id','gender','age','occupation','zip']
#标签
target = ['rating']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] =lbe.fit_transform(data[feature])

In [13]:
# 计算每个特征中不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature,data[feature].nunique()) for feature in sparse_features]
# print(fixlen_feature_columns)
#FM特征
linear_feature_columns = fixlen_feature_columns
# print(linear_feature_columns)
#DNN特征
dnn_feature_columns = fixlen_feature_columns
# print(dnn_feature_columns)
#特征名称
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
# print(feature_names)

In [14]:
# 将数据集切分为训练集和测试集
train,test = train_test_split(data,test_size = 0.2,random_state = 33)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [15]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns,dnn_feature_columns,task = 'regression')
model.compile('adam','mse',metrics = ['mse'])
history = model.fit(train_model_input,train[target].values,batch_size = 256,epochs =10,verbose =True,validation_split = 0.2)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1/1 [==============================] - 0s 343ms/step - loss: 13.8202 - mse: 13.8202 - val_loss: 13.9472 - val_mse: 13.9472
Epoch 2/10
1/1 [==============================] - 0s 20ms/step - loss: 13.6839 - mse: 13.6839 - val_loss: 13.8262 - val_mse: 13.8262
Epoch 3/10
1/1 [==============================] - 0s 20ms/step - loss: 13.5391 - mse: 13.5391 - val_loss: 13.6966 - val_mse: 13.6966
Epoch 4/10
1/1 [==============================] - 0s 20ms/step - loss: 13.3843 - mse: 13.3843 - val_loss: 13.5584 - val_mse: 13.5584
Epoch 5/10
1/1 [==============================] - 0s 26ms/step - loss: 13.2199 - mse: 13.2199 - val_loss: 13.4113 - val_mse: 13.4113
Epoch 6/10
1/1 [==============================] - 0s 21ms/step - loss: 13.0453 - mse: 13.0453 - val_loss: 13.2551 - val_mse: 13.2551
Epoch 7/10
1/1 [==============================] - 0s 20ms/step - loss: 12.8595 - mse: 12.8595 - val_loss: 13.0894 - val_mse: 13.0894
Epoch 8/10
1/1 [==============================] - 0s 20ms/step - loss: 12.6622 

In [16]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input,batch_size = 256)

In [18]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values,pred_ans),4)
rmse = round(mse ** 0.5,4)
print('test RMSE',rmse)

test RMSE 3.6842


In [19]:
# 加载完整数据
users = pd.read_csv('./users.dat',sep = '::',header = None)
movies = pd.read_csv('./movies.dat',sep = '::',header = None)
ratings = pd.read_csv('./ratings.dat',sep = '::',header = None)

users.columns = ['user_id','gender','age','occupation','zip']
movies.columns = ['movie_id','title','genres']
ratings.columns = ['user_id','movie_id','rating','timestamp']
#将3个源文件的数据融合
complete_data = users.merge(ratings,on = 'user_id').merge(movies,on='movie_id')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  afte

In [20]:
# 对特征标签进行编码
for feature in sparse_features:
    complete_data[feature] = lbe.fit_transform(complete_data[feature])

In [21]:
# 计算每个特征中不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature,complete_data[feature].nunique()) for feature in sparse_features]
# print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [22]:
# 将数据集切分为训练集和测试集
com_train,com_test = train_test_split(complete_data,test_size = 0.2,random_state = 33)
com_train_model_input = {name:com_train[name].values for name in feature_names}
com_test_model_input = {name:com_test[name].values for name in feature_names}

In [23]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns,dnn_feature_columns,task = 'regression')
model.compile('adam','mse',metrics = ['mse'])
model.fit(com_train_model_input,com_train[target].values,batch_size = 256,epochs =10,verbose =True,validation_split = 0.2)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2501/2501 [==============================] - 13s 5ms/step - loss: 1.0041 - mse: 1.0034 - val_loss: 0.8226 - val_mse: 0.8214
Epoch 2/10
2501/2501 [==============================] - 13s 5ms/step - loss: 0.8116 - mse: 0.8099 - val_loss: 0.8040 - val_mse: 0.8017
Epoch 3/10
2501/2501 [==============================] - 12s 5ms/step - loss: 0.7857 - mse: 0.7827 - val_loss: 0.7857 - val_mse: 0.7821
Epoch 4/10
2501/2501 [==============================] - 12s 5ms/step - loss: 0.7683 - mse: 0.7641 - val_loss: 0.7827 - val_mse: 0.7779
Epoch 5/10
2501/2501 [==============================] - 12s 5ms/step - loss: 0.7575 - mse: 0.7521 - val_loss: 0.7792 - val_mse: 0.7733
Epoch 6/10
2501/2501 [==============================] - 12s 5ms/step - loss: 0.7505 - mse: 0.7440 - val_loss: 0.7797 - val_mse: 0.7727
Epoch 7/10
2501/2501 [==============================] - 12s 5ms/step - loss: 0.7450 - mse: 0.7375 - val_loss: 0.7801 - val_mse: 0.7722
Epoch 8/10
2501/2501 [==============================] - 13s 5ms/st

In [24]:
#使用训练好模型进行预测
com_pred_ans = model.predict(com_test_model_input,batch_size = 256)

In [26]:
# 输出RMSE或MSE
mse = round(mean_squared_error(com_test[target].values,com_pred_ans),4)
rmse = round(mse ** 0.5,4)
print('test RMSE',rmse)

test RMSE 0.8773
